In [1]:
import folium
import ee

### Initialise Earth Engine

For installation instructions, check the [documentation](https://developers.google.com/earth-engine/python_install).

In [2]:
ee.Initialize()

#### Helper functions

In [3]:
# Define style for the difference map.
# Red -100% difference; Blue +100% difference
style_diff = '''
<RasterSymbolizer>\
  <ColorMap  type="ramp" extended="false" >
    <ColorMapEntry color="#ff0000" quantity="-1" label="0"/>
    <ColorMapEntry color="#555555" quantity="0" label="1"/>
    <ColorMapEntry color="#0000ff" quantity="1" label="2"/>
  </ColorMap>
</RasterSymbolizer>'''

def show_map(img):
    '''Retrieve map info and display an interactive map.'''
    d = img.getMapId()
    url = 'https://earthengine.googleapis.com/map/{}/{{z}}/{{x}}/{{y}}?token={}'.format(d['mapid'], d['token'])
    m = folium.Map(tiles='OpenStreetMap', location=[-33, 148], zoom_start=6)
    attr = 'Map data &copy; <a href="mailto:jose.padarian@sydney.edu.au">Jos&eacute; Padarian</a> </br><a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0;" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a </br><a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike</br>4.0 International License</a>.'
    layer = folium.TileLayer(url, attr=attr)
    layer.add_to(m)
    return m

def select_clear_pixels(img):
    '''Mask clouds, shadows, etc.'''
    mask = ee.Image(img).select('pixel_qa').bitwiseAnd(2).neq(0)
    return ee.Image(img).updateMask(mask)

def get_ndvi(year):
    '''Get NDVI image for the specified year'''
    date_ini = '{}-06-01'.format(year)
    date_end = '{}-08-15'.format(year)
    col = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(date_ini, date_end).filterBounds(nsw)
    ndvi = ee.ImageCollection(col.map(select_clear_pixels)).median().normalizedDifference(['B5', 'B4'])
    return ndvi

In [4]:
# Load NSW bondaries
nsw = ee.FeatureCollection('users/spadarian/boundaries/NSW_simple')

In [5]:
ndvi_2018 = get_ndvi(2018)
past_ndvi = ee.Image.cat([get_ndvi(y) for y in range(2014, 2017 + 1)]).reduce('mean')
diff = ndvi_2018.subtract(past_ndvi).divide(past_ndvi).clip(nsw).sldStyle(style_diff)

In [6]:
show_map(diff)